# Using FXCM’s REST API to automate your strategies using Python.

You can find documentation and tutorials for other languages by visiting FXCM’s Github.

Once open, search the ‘environments’ tab to find out if the necessary packages have already been installed. 

Requests and Pandas should already be installed, but the SocketIO library is to be installed separately: 
launch the Anaconda Prompt and type hitting enter in the end:

    pip install -U socketIO-client

To get an API token you will need an FXCM account: 
1. on  www.fxcm.com register for a free practice account by clicking Try Demo
2. complete the registration form with your information and 
3. click Trade Now to generate your practice account credentials, which will be sent to you via email. 
4. navigate to https://tradingstation.fxcm.com/ and enter your newly created login and password. 
5. once the platform has loaded, locate the button showing your login ID in the upper right corner. 
6. click this button and then click on token management and enter your password once again. 
7. clicking on generate token will create your authentication token for the REST API.

Open a Jupyter Notebook through the Anaconda Prompt:
"jupyter notebook" 

In [ ]:
import pandas as pd
import json
import requests
from socketIO_client import SocketIO

In [ ]:
# define our static connection information.  

TRADING_API_URL = 'https://api-demo.fxcm.com:443' # live environment address is in our REST API doc
WEBSOCKET_PORT = 443 # our websocket port
ACCESS_TOKEN = '3a047ccac7653cf794ad8622b89fa1926e0e05df'

In [ ]:
# Definitions for SocketIO events

# creation of 2 methods
def on_connect():
    print('Websocket Connected: ' + socketIO._engineIO_session.id)

def on_close():
        print('Websocket Closed.')

# initialize SocketIO
socketIO =  SocketIO(TRADING_API_URL, WEBSOCKET_PORT, params={'access_token' : ACCESS_TOKEN})

socketIO.on('connect', on_connect)
socketIO.on('disconnect', on_close)

bearer_access_token = "Bearer " + socketIO._engineIO_session.id + ACCESS_TOKEN

In [ ]:
# If we were assigned a bearer access token => we have successfully authenticated! 
# It should end up looking like this: Bearer nxdwUTW_ylimTxEqABi13a047ccac7653cf794ad8622b89fa1926e0e05df

print(bearer_access_token) # see if we were successful  

## Pulling Historical Pricing

In [ ]:
specifications for types of data: https://github.com/fxcm/RestAPI
Now we’re going to pull candle data, which means historical data, 
we’re using 1 which is the code for EUR/USD, and H1 stands for “tick frequency.

hist_response = requests.get( # send our request using the “GET” request
                             TRADING_API_URL + method, # specify the type of data we want to pull
                             headers = {
                                 'User-Agent': 'request',
                                 'Authorization': bearer_access_token,
                                 'Accept': 'application/json',
                                 'Content-Type': 'application/x-wwww-form-urlencoded'
                             },
                             params = {
                                 'num': 1000,
                                 'from': 1494086400,
                                 'to': 1503835200
                             })

if hist_response.status_code == 200: # response of “200” means success  
    print ("Data retrieved...")
    hist_data = hist_response.json() # use json to decode the information sent back to us
    candle_data = hist_data['candles']
    df = pd.DataFrame(candle_data)
    df.columns = ["time",
                  "bidopen", "bidclose", "bidhigh", "bidlow", 
                  "askopen", "askclose", "askhigh", "asklow", 
                  "TickQty"]
    df['time'] = pd.to_datetime(df['time'], unit='s') # column TIME goes in Epoch time, so we need to convert
    print(df)
else: # response of “400” means error  
    print(hist_response)

## Streaming Real-Time Prices

In [ ]:
method = '/subscribe'

sub_response = requests.post( # this time is post request
                             TRADING_API_URL + method,
                             headers = {
                                 'User-Agent': 'request',
                                 'Authorization': bearer_access_token,
                                 'Accept': 'application/json',
                                 'Content-Type': 'application/x-www-form-urlencoded'
                             },
                             data = { # instead of sending parameters ...
                                 'pairs': 'EUR/USD' # ... sending the currency pair we want to subscribe to
                             })

print(sub_response)
print(sub_response.json())



Pulling a single rate at a time can work; however, it’s not ideal.  


def on_price_update(msg): # for getting updates as soon as they’re available
    response = json.loads(msg)
    print(response)
   
if sub_response.status_code == 200:
    socketIO.on('EUR/USD', on_price_update) # create a subscription
    socketIO.wait() # subscribe to EUR/USD ticks

# fxcmpy 

https://www.fxcm.com/fxcmpy/index.html

Python wrapper: connecting to and using FXCM’s REST API 

pip install fxcmpy # all dependencies of the wrapper will automatically be installed
pip install fxcmpy –upgrade # regularly check for new versions 


In order to connect to the FXCM REST API via the fxcmpy Python wrapper, 
we will need to define a few settings about the connection and our credentials for logging in. 
To create a configuration file, we can utilize a simple text editor (such as notepad on Windows) 
and enter the following information:

[FXCM]
log_level = error
log_file = PATH_TO_AND_NAME_OF_LOG_FILE
access_token = YOUR_FXCM_API_TOKEN

For this article, we will save this file as fxcm.cfg 
and will save it in the same directory as our main program 
so that we only need to pass the fxcm.cfg file name as a parameter when connecting to REST API.

In [ ]:
import fxcmpy 
con = fxcmpy.fxcmpy(config_file='fxcm.cfg', server='demo') # for live account: ‘real’

con.get_instruments() # view all instruments that you can subscribe to on your account
con.get_instruments_for_candles() # instruments that you are currently subscribed to 

data = con.get_candles('GBP/JPY', period='D1') 
# Available periods for pulling historical data are: 
# ‘m1’, ‘m5’, ‘m15’, ‘m30’, ‘H1’, ‘H2’, ‘H3’, ‘H4’, ‘H6’, ‘H8′,’D1’, ‘W1’, or ‘M1’

data = con.get_candles('EUR/USD', period = 'm1', number = 5)

import datetime  as dt
start = dt.datetime(2018, 1, 1)
stop = dt.datetime(2018, 1, 30)
data = con.get_candles('EUR/USD', period = 'D1', start = start, stop = stop)

con.subscribe_market_data('EUR/USD')
data = con.get_prices('EUR/USD') # Streaming real-time prices
con.unsubscribe_market_data('EUR/USD') # unsubscribe
 

con.get_open_positions()# information on existing orders
con.create_market_buy_order('USD/JPY', (10) # instrument and the size of the order (in micro lots) 
con.create_market_sell_order('EUR/USD', (100)


# Good-Til-Canceled market order to buy 10k worth of GBP/JPY with a limit at the market price of 150.00:
con.open_trade(symbol = 'GBP/JPY', 
               is_buy = True, 
               rate = 105, 
               is_in_pips = False, 
               amount = 10,
               time_in_force = 'GTC', 
               order_type = 'AtMarket',
               limit = 150)

con.create_entry_order(symbol = 'GBP/JPY', 
                       is_buy = True, 
                       amount = 10, 
                       limit = 150, 
                       is_in_pips = False, 
                       time_in_force = 'GTC',
                       rate = 105)


id = con.get_order_ids()[-1]
con.change_order_stop_limit(order_id = id, limit = 140)
con.change_order(order_id = id, amount = 100, rate = 110)

con.delete_order(id)

pos = con.get_open_position(id) # close a single position
pos.close()

con.close_all_for_symbol('GBP/JPY')

con.close_all()

# EMA Crossover strategy

https://github.com/fxcm/RestAPI/blob/master/Python-Backtest_Examples/EMA%20Crossover%20Strategy%20backtest.py

In [ ]:
import fxcmpy 
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
from pyti.exponential_moving_average import exponential_moving_average as ema

con = fxcmpy.fxcmpy(config_file='fxcm.cfg') # file contains settings to connect to the FXCM REST API 

df = con.get_candles('GBP/JPY', period = 'D1', 
                     start = dt.datetime(2016, 1, 1), 
                     end = dt.datetime(2018, 6, 10))

df['difference (pips)'] = (df['askclose'] - df['askopen']) * 100 # convert the daily returns into pips while JPY

ema_fast = 12
ema_slow = 20
df['ema_fast'] = ema(df['askclose'], ema_fast) # from the pyti library
df['ema_slow'] = ema(df['askclose'], ema_slow)

# When the Fast EMA crosses above the Slow EMA a buy order will be triggered
df['position'] = np.where(df['ema_fast'] > df['ema_slow'],1,0) 
df['signal'] = df['position'].diff()


pip_cost = .0879
lot_size = 10
Returns = 0
CountPL=False
for i, row in df.iterrows():
    if CountPL==True:
        returns += (row['difference (pips)'] * pip_cost * lot_size)
        df.loc[i,'total'] = returns
    else:
        df.loc[i,'total'] = returns
 
    if row['position'] == 1:
        CountPL=True
    else:
        CountPL=False

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(111,  ylabel='GBP/JPY Price')
df['ask_close'].plot(ax=ax1, color='r', lw=1)
df[['ema_fast','ema_slow']].plot(ax=ax1, lw=2)
ax1.plot(df.loc[df.position == 1.0].index, 
         df.ema_fast[df.position == 1.0],
         '^', markersize=10, color='m' )
ax1.plot(df.loc[df.position == -1.0].index, 
         df.ema_slow[df.position == -1.0],
         'v', markersize=10, color='k')
ax2 = ax1.twinx()
ax2.set_ylabel('Profits in $')
ax2.plot(df['total'], color = 'green')
plt.show()